# From Monte Carlo ensemble to most‑likely basins

This notebook takes the stack of Monte Carlo basin realisations
(`basins_mc_*.tif`) and derives a **single, most‑likely basin map**
together with several uncertainty diagnostics.

The outputs are:

- `basins_most_likely.tif` – segmentation where each pixel is assigned the basin
  label that occurs most frequently across all Monte Carlo runs.
- `basins_certainty.tif` – fraction `n_max / N` for that most‑likely label at each pixel
  (values near 1 are very certain, values near 0.5 or lower indicate ambiguous pixels).
- `basins_uncertainty.tif` – simply `1 − certainty`, a more intuitive “uncertainty” map.
- `basin_boundary_probability.tif` – probability that a pixel behaves as a
  **basin boundary** across the ensemble.
- `basin_stable_divides.tif` / `basin_uncertain_divides.tif` – binary masks that
  classify divides as stable or uncertain based on boundary probability thresholds.


## 1. Compute most‑likely basins and boundary probability

This cell:

- Loads all `basins_mc_*.tif` files from the Monte Carlo directory.
- Chooses one realisation as a **reference grid** (only for geometry / domain).
- For each pixel, counts how often each basin ID occurs across the ensemble.
- Determines the **winning label** (the most frequent basin ID) and its count `n_max`.
- Writes:

  * `basins_most_likely.tif` – map of winning labels.
  * `basins_certainty.tif` – `n_max / N` per pixel.
  * `basin_boundary_probability.tif` – fraction of runs in which a pixel lies on a
    basin boundary (i.e. neighbouring pixels carry different winning labels).

Chunked reading/writing is used so that the computation works efficiently
on large Greenland‑scale rasters.


In [2]:
import numpy as np
import rasterio
from rasterio.windows import Window
from pathlib import Path
from collections import defaultdict

# =============================================================================
# USER SETTINGS
# =============================================================================
MC_DIR = Path(r"E:\Rasmus\DTU\Cryo\4DGreenland\Basins_serious\prodem_19_MC")
BASIN_PATTERN = "basins_mc_*.tif"

# Reference realisation is ONLY used as a label grid / domain
REF_INDEX = 0   # 0 = first in sorted list

# Outputs
OUT_MOST_LIKELY = MC_DIR / "basins_most_likely.tif"
OUT_CERT        = MC_DIR / "basins_certainty.tif"
OUT_UNCERT      = MC_DIR / "basins_uncertainty.tif"
OUT_BOUND_PROB  = MC_DIR / "basin_boundary_probability.tif"
OUT_STABLE_DIV  = MC_DIR / "basin_stable_divides.tif"
OUT_UNCERT_DIV  = MC_DIR / "basin_uncertain_divides.tif"

# Chunking
CHUNK_ROWS = 32

# Thresholds
P_STABLE_PIXEL = 0.90   # pixelwise boundary probability for stable/uncertain divides (map 3)
P_STABLE_EDGE  = 0.70   # mean boundary probability along a divide to keep that divide
P_MIN_DIV      = 0.00   # min prob to be considered "any divide"

# =============================================================================
# HELPER: max count per pixel (ignoring label 0)
# =============================================================================
def max_count_nonzero(stack_2d):
    """
    stack_2d: (N_runs, M) with integer labels (0 = background).
    Returns max_counts (M,) = largest count of any non-zero label per pixel.
    """
    N, M = stack_2d.shape
    s = np.sort(stack_2d, axis=0)  # (N, M)

    change = np.vstack([
        np.ones((1, M), dtype=bool),
        s[1:, :] != s[:-1, :]
    ])
    g = np.cumsum(change, axis=0).astype(np.int32)

    Np1 = N + 1
    col_offsets = (np.arange(M, dtype=np.int64) * Np1)[None, :]
    global_group = g.astype(np.int64) + col_offsets
    flat_group = global_group.ravel()

    max_gid = int(flat_group.max())
    counts = np.bincount(flat_group, minlength=max_gid + 1)

    group_size = counts[flat_group].reshape(N, M)

    positive = s > 0
    group_size_pos = group_size * positive

    max_counts = group_size_pos.max(axis=0)
    return max_counts.astype(np.int32)

# =============================================================================
# 0) COLLECT FILES & BASIC INFO
# =============================================================================
basin_files = sorted(MC_DIR.glob(BASIN_PATTERN))
if not basin_files:
    raise FileNotFoundError(f"No files matching {BASIN_PATTERN} in {MC_DIR}")

N_runs = len(basin_files)
print(f"Found {N_runs} basin realisations")

ref_file = basin_files[REF_INDEX]
print(f"Using {ref_file.name} as label grid / domain")

with rasterio.open(ref_file) as src_ref:
    meta = src_ref.meta.copy()
    width, height = src_ref.width, src_ref.height
    transform = src_ref.transform
    crs = src_ref.crs
    nodata_ref = src_ref.nodata if src_ref.nodata is not None else 0

    max_ref_label = 0
    for row_start in range(0, height, CHUNK_ROWS):
        row_stop = min(row_start + CHUNK_ROWS, height)
        window = Window(0, row_start, width, row_stop - row_start)
        ref_chunk = src_ref.read(1, window=window)
        max_ref_label = max(max_ref_label, int(ref_chunk.max()))

print("Max reference basin ID:", max_ref_label)

# =============================================================================
# 1) BUILD PER-RUN MAPPINGS → REFERENCE BASINS
# =============================================================================
maps = []  # map_r[label_in_run] = label_in_ref

src_ref_global = rasterio.open(ref_file)

for idx, fpath in enumerate(basin_files):
    print(f"\nMapping run {idx+1}/{N_runs}: {fpath.name}")

    if idx == REF_INDEX:
        # identity mapping for reference
        map_r = np.arange(max_ref_label + 1, dtype=np.int32)
        maps.append(map_r)
        continue

    mapping_counts = defaultdict(int)

    with rasterio.open(fpath) as src_run:
        nodata_run = src_run.nodata if src_run.nodata is not None else 0

        for row_start in range(0, height, CHUNK_ROWS):
            row_stop = min(row_start + CHUNK_ROWS, height)
            window = Window(0, row_start, width, row_stop - row_start)

            ref_chunk = src_ref_global.read(1, window=window)
            run_chunk = src_run.read(1, window=window)

            domain_chunk = (ref_chunk != 0) & (ref_chunk != nodata_ref)
            mask = domain_chunk & (run_chunk != 0) & (run_chunk != nodata_run)
            if not np.any(mask):
                continue

            rvals = run_chunk[mask].astype(np.int64)
            kvals = ref_chunk[mask].astype(np.int64)

            idx_comb = rvals * (max_ref_label + 1) + kvals
            uniq, counts = np.unique(idx_comb, return_counts=True)
            for u, c in zip(uniq, counts):
                mapping_counts[int(u)] += int(c)

    if not mapping_counts:
        print("  Warning: no overlaps; mapping everything to 0")
        # auto-size mapping
        with rasterio.open(fpath) as src_run:
            max_run_label = 0
            for row_start in range(0, height, CHUNK_ROWS):
                row_stop = min(row_start + CHUNK_ROWS, height)
                window = Window(0, row_start, width, row_stop - row_start)
                run_chunk = src_run.read(1, window=window)
                max_run_label = max(max_run_label, int(run_chunk.max()))
        map_r = np.zeros(max_run_label + 1, dtype=np.int32)
        maps.append(map_r)
        continue

    max_run_label = max(k // (max_ref_label + 1) for k in mapping_counts.keys())
    best_k = np.zeros(max_run_label + 1, dtype=np.int32)
    best_n = np.zeros(max_run_label + 1, dtype=np.int64)

    for idx_comb, c in mapping_counts.items():
        j = idx_comb // (max_ref_label + 1)  # label in run
        if j == 0:
            continue
        k = idx_comb % (max_ref_label + 1)   # label in ref
        if c > best_n[j]:
            best_n[j] = c
            best_k[j] = k

    map_r = best_k
    map_r[0] = 0
    maps.append(map_r)

src_ref_global.close()
print("\n✓ Finished building label mappings to reference basins")

# =============================================================================
# 2) PASS 1: boundary probabilities for ref divides + divide stability per pair
# =============================================================================
sum_prob = defaultdict(float)   # sum of boundary prob along each ref basin pair
count    = defaultdict(int)     # number of samples per pair

with rasterio.open(ref_file) as src_ref:
    for row_start in range(0, height, CHUNK_ROWS):
        row_stop = min(row_start + CHUNK_ROWS, height)
        nrows_chunk = row_stop - row_start
        print(f"[Pass 1] Rows {row_start}–{row_stop-1}")

        window = Window(0, row_start, width, nrows_chunk)
        ref_chunk = src_ref.read(1, window=window)

        domain_chunk = (ref_chunk != 0) & (ref_chunk != nodata_ref)

        # stack of mapped REF labels (N_runs, nrows, width)
        stack_ref = np.zeros((N_runs, nrows_chunk, width), dtype=np.int32)

        for i, fpath in enumerate(basin_files):
            map_r = maps[i]
            with rasterio.open(fpath) as src_run:
                nodata_run = src_run.nodata if src_run.nodata is not None else 0
                run_chunk = src_run.read(1, window=window)

            run_chunk = run_chunk.astype(np.int32)
            run_chunk[run_chunk == nodata_run] = 0
            run_chunk[run_chunk < 0] = 0
            mask_big = run_chunk >= len(map_r)
            run_chunk[mask_big] = 0

            mapped = map_r[run_chunk]
            mapped[~domain_chunk] = 0
            stack_ref[i, :, :] = mapped

        # Boundary indicator per run
        labels = stack_ref
        valid = labels != 0
        B = np.zeros_like(labels, dtype=bool)

        # N-S
        diff_ns = labels[:, 1:, :] != labels[:, :-1, :]
        valid_ns = valid[:, 1:, :] & valid[:, :-1, :]
        bd_ns = diff_ns & valid_ns
        B[:, 1:, :] |= bd_ns
        B[:, :-1, :] |= bd_ns

        # E-W
        diff_ew = labels[:, :, 1:] != labels[:, :, :-1]
        valid_ew = valid[:, :, 1:] & valid[:, :, :-1]
        bd_ew = diff_ew & valid_ew
        B[:, :, 1:] |= bd_ew
        B[:, :, :-1] |= bd_ew

        bprob_chunk = B.sum(axis=0).astype(np.float32) / float(N_runs)
        bprob_chunk[~domain_chunk] = 0.0

        # Accumulate boundary stability for each pair of reference basins
        # using reference labels + bprob_chunk
        h, w = ref_chunk.shape
        for i_row in range(h):
            for j_col in range(w):
                a = ref_chunk[i_row, j_col]
                if a == 0 or a == nodata_ref:
                    continue

                # east neighbour
                if j_col + 1 < w:
                    b = ref_chunk[i_row, j_col + 1]
                    if b != a and b != 0 and b != nodata_ref:
                        key = (a, b) if a < b else (b, a)
                        val = 0.5 * (bprob_chunk[i_row, j_col] +
                                     bprob_chunk[i_row, j_col + 1])
                        sum_prob[key] += float(val)
                        count[key] += 1

                # south neighbour
                if i_row + 1 < h:
                    b = ref_chunk[i_row + 1, j_col]
                    if b != a and b != 0 and b != nodata_ref:
                        key = (a, b) if a < b else (b, a)
                        val = 0.5 * (bprob_chunk[i_row, j_col] +
                                     bprob_chunk[i_row + 1, j_col])
                        sum_prob[key] += float(val)
                        count[key] += 1

print("✓ Finished Pass 1: collected divide stability statistics")

# =============================================================================
# 3) MERGE BASINS ACROSS UNSTABLE DIVIDES → MOST-LIKELY SEGMENTATION
# =============================================================================
max_label = max_ref_label
parent = np.arange(max_label + 1, dtype=np.int32)

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

# Merge across edges with low mean probability
print("Merging basins across weak divides ...")
for (a, b), total in sum_prob.items():
    c = count[(a, b)]
    if c == 0:
        continue
    mean_prob = total / float(c)
    if mean_prob < P_STABLE_EDGE:
        union(a, b)

# Compress to new labels 1..K
root_to_new = {}
mapping_ref_to_cluster = np.zeros(max_label + 1, dtype=np.int32)
next_label = 1
for old in range(1, max_label + 1):
    if old == nodata_ref:
        mapping_ref_to_cluster[old] = 0
        continue
    root = find(old)
    if root not in root_to_new:
        root_to_new[root] = next_label
        next_label += 1
    mapping_ref_to_cluster[old] = root_to_new[root]
mapping_ref_to_cluster[0] = 0

n_clusters = next_label - 1
print(f"Most-likely segmentation has {n_clusters} basins")

# =============================================================================
# 4) PASS 2: write MOST-LIKELY basins + certainty + boundary prob
# =============================================================================
meta_i32 = meta.copy()
meta_i32.update(dtype="int32", nodata=0, count=1, compress="LZW")
meta_f32 = meta.copy()
meta_f32.update(dtype="float32", nodata=0.0, count=1, compress="LZW")

with rasterio.open(ref_file) as src_ref, \
     rasterio.open(OUT_MOST_LIKELY, "w", **meta_i32) as dst_seg, \
     rasterio.open(OUT_CERT, "w", **meta_f32) as dst_cert, \
     rasterio.open(OUT_BOUND_PROB, "w", **meta_f32) as dst_bprob:

    for row_start in range(0, height, CHUNK_ROWS):
        row_stop = min(row_start + CHUNK_ROWS, height)
        nrows_chunk = row_stop - row_start
        print(f"[Pass 2] Rows {row_start}–{row_stop-1}")

        window = Window(0, row_start, width, nrows_chunk)
        ref_chunk = src_ref.read(1, window=window).astype(np.int32)

        # Domain where reference has a basin
        domain_chunk = (ref_chunk != 0) & (ref_chunk != nodata_ref)

        # Prepare safe index array: nodata / negatives → 0
        ref_idx = ref_chunk.copy()
        ref_idx[~domain_chunk] = 0
        ref_idx[ref_idx < 0] = 0

        # Most-likely segmentation for this chunk (union of ref basins)
        seg_chunk = mapping_ref_to_cluster[ref_idx]
        seg_chunk[~domain_chunk] = 0
        dst_seg.write(seg_chunk.astype(np.int32), 1, window=window)



        # stack of FINAL labels (mapped→ref→cluster)
        stack_final = np.zeros((N_runs, nrows_chunk, width), dtype=np.int32)

        for i, fpath in enumerate(basin_files):
            map_r = maps[i]
            with rasterio.open(fpath) as src_run:
                nodata_run = src_run.nodata if src_run.nodata is not None else 0
                run_chunk = src_run.read(1, window=window)

            run_chunk = run_chunk.astype(np.int32)
            run_chunk[run_chunk == nodata_run] = 0
            run_chunk[run_chunk < 0] = 0
            mask_big = run_chunk >= len(map_r)
            run_chunk[mask_big] = 0

            labels_ref = map_r[run_chunk]
            labels_ref[~domain_chunk] = 0
            labels_final = mapping_ref_to_cluster[labels_ref]
            stack_final[i, :, :] = labels_final

        # ---- pixel-wise certainty (n_max/N) ----
        N = N_runs
        M = nrows_chunk * width
        stack_flat = stack_final.reshape(N, M)
        max_counts = max_count_nonzero(stack_flat)     # (M,)
        certainty_flat = max_counts.astype(np.float32) / float(N)
        cert_chunk = certainty_flat.reshape(nrows_chunk, width)
        cert_chunk[~domain_chunk] = 0.0

        # ---- boundary probability w.r.t final segmentation ----
        labels = stack_final
        valid = labels != 0
        B = np.zeros_like(labels, dtype=bool)

        # N-S
        diff_ns = labels[:, 1:, :] != labels[:, :-1, :]
        valid_ns = valid[:, 1:, :] & valid[:, :-1, :]
        bd_ns = diff_ns & valid_ns
        B[:, 1:, :] |= bd_ns
        B[:, :-1, :] |= bd_ns

        # E-W
        diff_ew = labels[:, :, 1:] != labels[:, :, :-1]
        valid_ew = valid[:, :, 1:] & valid[:, :, :-1]
        bd_ew = diff_ew & valid_ew
        B[:, :, 1:] |= bd_ew
        B[:, :, :-1] |= bd_ew

        bprob_chunk = B.sum(axis=0).astype(np.float32) / float(N)
        bprob_chunk[~domain_chunk] = 0.0

        dst_cert.write(cert_chunk, 1, window=window)
        dst_bprob.write(bprob_chunk, 1, window=window)

print("✓ Wrote basins_most_likely, basins_certainty, basin_boundary_probability")

# =============================================================================
# 5) DERIVED PRODUCTS: UNCERTAINTY + DIVIDE MASKS
# =============================================================================
with rasterio.open(OUT_CERT) as src_c:
    cert = src_c.read(1).astype(np.float32)
    meta_u = src_c.meta.copy()

uncert = 1.0 - cert
uncert[uncert < 0] = 0.0

meta_u.update(dtype="float32", nodata=0.0, count=1, compress="LZW")
with rasterio.open(OUT_UNCERT, "w", **meta_u) as dst:
    dst.write(uncert, 1)

print("✓ Wrote basins_uncertainty:", OUT_UNCERT.name)

with rasterio.open(OUT_BOUND_PROB) as src_bp:
    bprob = src_bp.read(1).astype(np.float32)
    meta_div = src_bp.meta.copy()

stable_divides    = (bprob >= P_STABLE_PIXEL).astype("uint8")
uncertain_divides = ((bprob >= P_MIN_DIV) & (bprob < P_STABLE_PIXEL)).astype("uint8")

meta_div.update(dtype="uint8", nodata=0, count=1, compress="LZW")

with rasterio.open(OUT_STABLE_DIV, "w", **meta_div) as dst:
    dst.write(stable_divides, 1)

with rasterio.open(OUT_UNCERT_DIV, "w", **meta_div) as dst:
    dst.write(uncertain_divides, 1)

print("✓ Wrote stable and uncertain divide masks")
print("Done.")


Found 38 basin realisations
Using basins_mc_001.tif as label grid / domain
Max reference basin ID: 7498

Mapping run 1/38: basins_mc_001.tif

Mapping run 2/38: basins_mc_002.tif

Mapping run 3/38: basins_mc_003.tif

Mapping run 4/38: basins_mc_004.tif

Mapping run 5/38: basins_mc_005.tif

Mapping run 6/38: basins_mc_006.tif

Mapping run 7/38: basins_mc_007.tif

Mapping run 8/38: basins_mc_008.tif

Mapping run 9/38: basins_mc_009.tif

Mapping run 10/38: basins_mc_010.tif

Mapping run 11/38: basins_mc_011.tif

Mapping run 12/38: basins_mc_012.tif

Mapping run 13/38: basins_mc_013.tif

Mapping run 14/38: basins_mc_014.tif

Mapping run 15/38: basins_mc_015.tif

Mapping run 16/38: basins_mc_016.tif

Mapping run 17/38: basins_mc_017.tif

Mapping run 18/38: basins_mc_018.tif

Mapping run 19/38: basins_mc_019.tif

Mapping run 20/38: basins_mc_020.tif

Mapping run 21/38: basins_mc_021.tif

Mapping run 22/38: basins_mc_022.tif

Mapping run 23/38: basins_mc_023.tif

Mapping run 24/38: basins_mc_0

## 2. Derived uncertainty products and divide masks

This cell derives additional products from the certainty and boundary‑probability maps:

- `basins_uncertainty.tif` is computed as `1 − basins_certainty`.
- Stable and uncertain divides are identified using two thresholds:

  * `P_STABLE_PIXEL` – pixels with boundary probability ≥ this value are marked
    as **stable divides**.
  * `P_MIN_DIV` – pixels with boundary probability between `P_MIN_DIV` and
    `P_STABLE_PIXEL` are marked as **uncertain divides**.

The result is two binary rasters:

- `basin_stable_divides.tif`
- `basin_uncertain_divides.tif`

that can be visualised on top of the most‑likely basin map to highlight areas
where the drainage structure is well constrained versus ambiguous.


In [3]:
OUT_NMAX_FRAC = MC_DIR / "basins_nmax_over_N.tif"
# After writing OUT_CERT and OUT_UNCERT
with rasterio.open(OUT_CERT) as src_c:
    cert = src_c.read(1).astype(np.float32)
    meta_n = src_c.meta.copy()

meta_n.update(dtype="float32", nodata=0.0, count=1, compress="LZW")
with rasterio.open(OUT_NMAX_FRAC, "w", **meta_n) as dst:
    dst.write(cert, 1)

print("✓ Wrote basins_nmax_over_N (same as basins_certainty):", OUT_NMAX_FRAC.name)


✓ Wrote basins_nmax_over_N (same as basins_certainty): basins_nmax_over_N.tif
